In [7]:
import numpy as np
import pandas as pd
import nltk
import re

In [8]:
df = pd.read_csv("tennis_articles_v4.csv")
df.head()
df['article_text'][0]
df['article_text'][1]

"BASEL, Switzerland (AP), Roger Federer advanced to the 14th Swiss Indoors final of his career by beating seventh-seeded Daniil Medvedev 6-1, 6-4 on Saturday. Seeking a ninth title at his hometown event, and a 99th overall, Federer will play 93th-ranked Marius Copil on Sunday. Federer dominated the 20th-ranked Medvedev and had his first match-point chance to break serve again at 5-1. He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand. He clinched on his fourth chance when Medvedev netted from the baseline. Copil upset expectations of a Federer final against Alexander Zverev in a 6-3, 6-7 (6), 6-4 win over the fifth-ranked German in the earlier semifinal. The Romanian aims for a first title after arriving at Basel without a career win over a top-10 opponent. Copil has two after also beating No. 6 Marin Cilic in the second round. Copil fired 26 aces past Zverev and never dropped serve, clinching after 2 1/2 hours with

In [9]:
#Splilt sentences
from nltk.tokenize import sent_tokenize
sentences = []
for s in df['article_text']:
  sentences.append(sent_tokenize(s))

sentences = [y for x in sentences for y in x] # flatten list

sentences[:5]

['Maria Sharapova has basically no friends as tennis players on the WTA Tour.',
 "The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much.",
 'I think everyone knows this is my job here.',
 "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.",
 "I'm a pretty competitive girl."]

In [10]:
#wget http://nlp.stanford.edu/data/glove.6B.zip
#unzip glove*.zip


In [11]:
#Let’s extract the words embeddings or word vectors.
word_embeddings = {}
f = open('glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [12]:
len(word_embeddings)

193442

In [13]:
#Text Preprocessing
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]

In [14]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [15]:
# function to remove stopwords
def remove_stopwords(sen):
    sen_new = " ".join([i for i in sen if i not in stop_words])
    return sen_new

In [19]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
print(clean_sentences)

['maria sharapova basically friends tennis players wta tour', 'russian player problems openly speaking recent interview said really hide feelings much', 'think everyone knows job', 'courts court playing competitor want beat every single person whether locker room across net one strike conversation weather know next minutes go try win tennis match', 'pretty competitive girl', 'say hellos sending players flowers well', 'uhm really friendly close many players', 'lot friends away courts', 'said really close lot players something strategic', 'different men tour women tour', '', 'think sport mean friends everyone categorized tennis player going get along tennis players', 'think every person different interests', 'friends completely different jobs interests met different parts life', 'think everyone thinks tennis players greatest friends', 'ultimately tennis small part', 'many things interested', 'basel switzerland ap roger federer advanced th swiss indoors final career beating seventh seeded

In [17]:
# Extract word vectors
word_embeddings = {}
f = open('glove.6B.50d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    word_embeddings[word] = coefs
f.close()

In [40]:
sentence_vectors = []
for i in clean_sentences:
  if len(i) != 0:
    v = sum([word_embeddings.get(w, np.zeros((50,))) for w in i.split()])/(len(i.split())+0.001)
  else:
    v = np.zeros((50,))
  sentence_vectors.append(v)

In [33]:
# similarity matrix
sim_mat = np.zeros([len(sentences), len(sentences)])
len(sentence_vectors)

119

In [23]:
from sklearn.metrics.pairwise import cosine_similarity

In [42]:
for i in range(len(sentences)):
  for j in range(len(sentences)):
    if i != j:
      sim_mat[i][j] = cosine_similarity(sentence_vectors[i].reshape(1,50), sentence_vectors[j].reshape(1,50))[0,0]

In [43]:
#Applying page rank algorithm
import networkx as nx

nx_graph = nx.from_numpy_array(sim_mat)
scores = nx.pagerank(nx_graph)


In [44]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)


In [46]:
# Extract top 10 sentences as the summary
for i in range(1):
  print(ranked_sentences[i][1])


Speaking at the Swiss Indoors tournament where he will play in Sundays final against Romanian qualifier Marius Copil, the world number three said that given the impossibly short time frame to make a decision, he opted out of any commitment.
